# Playground - data flow graph 

In [1]:
import os
import networkx as nx
import pyvis as pv
import matplotlib.pyplot as plt
import graphviz as gv
from typing import Iterable, Tuple, Set, List, Dict, OrderedDict, Optional
import bril_model as bm
from bril_model.form_blocks import print_blocks, form_blocks

SAVE_DIR = "./save"

# DEMO_BRIL_FILE = "./example/in_class_example_1.bril"
# DEMO_BRIL_FILE = "./example/in_class_example_2.bril"
# DEMO_BRIL_FILE = "./example/in_class_example_3.bril"
# DEMO_BRIL_FILE = "../bril/benchmarks/core/is-decreasing.bril"
# DEMO_BRIL_FILE = "../bril/benchmarks/core/hanoi.bril"
# DEMO_BRIL_FILE = "../bril/benchmarks/core/birthday.bril"
# DEMO_BRIL_FILE = "../bril/examples/test/df/fact.bril"
# DEMO_BRIL_FILE = "../bril/examples/test/df/cond.bril"
# DEMO_BRIL_FILE = "../bril/examples/test/df/cond-args.bril"
DEMO_BRIL_FILE = "../bril/examples/test/dom/loopcond.bril"
# DEMO_BRIL_FILE = "../bril/examples/test/dom/while.bril"

In [2]:
ENTRY_POINT_NAME = 'ENTRY'
RETURN_POINT_NAME = 'RETURN'

In [3]:
class BasicBlock():
    def __init__(self, label: bm.BrilInstruction_Label, instrs: List[bm.BrilInstruction]):
        if not isinstance(label, bm.BrilInstruction_Label):
            raise TypeError(f"Expected label, got {type(label)}")
        if label not in instrs:
            instrs.insert(0, label)
        self.instrs = instrs
        self.succ: Set[BasicBlock] = set()
        self.pred: Set[BasicBlock] = set()
    
    @property
    def label(self):
        return self.instrs[0]
    
    def __hash__(self) -> int:
        return hash(self.label)
    def __eq__(self, o: object) -> bool:
        if not isinstance(o, BasicBlock):
            return False
        return self.label == o.label
    def __lt__(self, o: object) -> bool:
        if not isinstance(o, BasicBlock):
            return False
        # return block is always the last one
        if self.label.label == RETURN_POINT_NAME: return False
        if o.label.label == RETURN_POINT_NAME: return True
        # entry block is always the first one
        if self.label.label == ENTRY_POINT_NAME: return True
        if o.label.label == ENTRY_POINT_NAME: return False
        return self.label < o.label

    def __str__(self):
        if self.label.label in {ENTRY_POINT_NAME, RETURN_POINT_NAME}: return self.label.label
        _instrs = '\n  '.join([x.to_briltxt() for x in self.instrs[1:]])
        if _instrs:
            _instrs = f"\n  {_instrs}"
        return f"{self.label.to_briltxt()}{_instrs}"
    def __repr__(self):
        return f"{self.__class__.__name__} :: {self.label.to_briltxt()}..[{len(self.instrs) - 1}]"

In [4]:
def iter_func_blocks(bs: bm.BrilScript) -> Iterable[Tuple[bm.BrilFunction, List[BasicBlock]]]:
    for each_func in bs.functions:
        bbs: List[BasicBlock] = list()
        anonymous_id = 0
        for each_block in form_blocks(each_func.instrs):
            this_block_label: bm.BrilInstruction_Label = None
            if isinstance(each_block[0], bm.BrilInstruction_Label):
                this_block_label = each_block[0]
            else:
                this_block_label = bm.BrilInstruction_Label(dict(label='_f{}._anon{}'.format(str(hash(each_func.name)%1000).zfill(3), anonymous_id)))
                anonymous_id += 1
            bbs.append(BasicBlock(this_block_label, each_block))
        yield (each_func, bbs)

def generate_func_cfg_dict(bscript: bm.BrilScript) -> Dict[bm.BrilFunction, List[BasicBlock]]:
    app_bbs_dict: OrderedDict[bm.BrilFunction, List[BasicBlock]] = {}
    for each_func, basic_blocks in iter_func_blocks(bscript):
        print("Function: {}".format(each_func.name))
        entry_bb = BasicBlock(bm.BrilInstruction_Label(dict(label=ENTRY_POINT_NAME)), [])
        return_bb = BasicBlock(bm.BrilInstruction_Label(dict(label=RETURN_POINT_NAME)), [])
        prev_bb: Optional[BasicBlock] = None
        is_first: bool = True
        for each_bb in basic_blocks:
            if prev_bb:
                # reason: fallthrough edge
                prev_bb.succ.add(each_bb)
                each_bb.pred.add(prev_bb)
            elif is_first:
                # reason: entry edge
                entry_bb.succ.add(each_bb)
                each_bb.pred.add(entry_bb)
                is_first = False
            
            # get last instruction if exists
            final_instr = each_bb.instrs[-1] if each_bb.instrs else None
            if final_instr is None:
                # empty block, skip
                prev_bb = each_bb
            elif final_instr.op in ['jmp', 'br']:
                # reaching control flow instruction
                for redirect_instr_to_bb_label in final_instr.labels:
                    next_bb = next((bb for bb in basic_blocks if bb.label.label == redirect_instr_to_bb_label), None)
                    if next_bb is None:
                        raise ValueError(f"Cannot find block with label {redirect_instr_to_bb_label}")
                    # reason: control flow edge
                    each_bb.succ.add(next_bb)
                    next_bb.pred.add(each_bb)
                prev_bb = None
            elif final_instr.op in ['ret']:
                # reaching return instruction
                # reason: return edge
                each_bb.succ.add(return_bb)
                return_bb.pred.add(each_bb)
                prev_bb = None
                # explicit return block, no fallthrough
            else:
                # normal instruction, prepare for fallthrough
                prev_bb = each_bb
        # check if last block has no fallthrough
        if prev_bb:
            # reason: return edge
            prev_bb.succ.add(return_bb)
            return_bb.pred.add(prev_bb)
        # add entry/return block
        basic_blocks.insert(0, entry_bb)
        basic_blocks.append(return_bb)
        app_bbs_dict[each_func] = basic_blocks
    return app_bbs_dict

def getBasicBlockByLabel(bb_list: List[BasicBlock], label: str) -> Optional[BasicBlock]:
    return next((bb for bb in bb_list if bb.label.label == label), None)

In [5]:
# load the bril script
bscript = bm.BrilScript(script_name=os.path.basename(DEMO_BRIL_FILE), file_dir=os.path.dirname(DEMO_BRIL_FILE))
print(bscript)
app_graph: Dict[bm.BrilFunction, List[BasicBlock]] = generate_func_cfg_dict(bscript)
print("Functions: {}".format(', '.join([f"[{idx}]={x.name}" for idx, x in enumerate(app_graph.keys())])))

BrilScript ::	loopcond.bril <1 func> @ ../bril/examples/test/dom
Function: main
Functions: [0]=main


In [6]:
def generate_dom_dict(bbs: List[BasicBlock]) -> dict[BasicBlock, set[BasicBlock]]:
    # Initialize dominator sets for each block
    # Initially, every block dominates every other block
    dom: Dict[BasicBlock, Set[BasicBlock]] = {bb: set(bbs) for bb in bbs}
    # The entry block only dominates itself
    bb_entry = getBasicBlockByLabel(bbs, ENTRY_POINT_NAME)
    dom[bb_entry] = set([bb_entry])

    changed: bool = True
    while changed:
        changed = False
        for each_bb in bbs:
            if each_bb == bb_entry: continue
            # The dominator set of a bb is the intersection of the dominator sets of its predecessors
            this_bb_dom = set.intersection(*[dom[each_pred] for each_pred in each_bb.pred])
            # The dominator set of a bb should includes itself
            this_bb_dom.add(each_bb)
            # Update if changed
            if dom[each_bb] != this_bb_dom:
                dom[each_bb] = this_bb_dom
                changed = True
    return dom

# get the last dom of a given block
def get_lease_superset_dom(dom_dict: dict[BasicBlock, set[BasicBlock]], bb: BasicBlock) -> Optional[BasicBlock]:
    return next((test_dom for test_dom in dom_dict[bb] if ((dom_dict[bb] - dom_dict[test_dom]) == set([bb]))), None)

# # get the dom frontiers of a given block
# # The easy and straight forward way to understand. 
# def get_dom_frontier(dom: Dict[BasicBlock, Set[BasicBlock]], bb: BasicBlock) -> Set[BasicBlock]:
#     df: Set[BasicBlock] = set()
#     for test_with_bb in dom.keys():
#         # skip if `A` does dominate `b`, except for `A` == `b`
#         if bb in dom[test_with_bb] and bb != test_with_bb: continue
#         # get all dominators of `b`'s predecessors
#         all_testbb_pred_dom: Set[BasicBlock] = set.union(*[dom[each_tbp] for each_tbp in test_with_bb.pred] or [set()])
#         # skip if `A` does not dominate a pred of `b`
#         if bb not in all_testbb_pred_dom: continue
#         # now, `a` does not dominate `b`, but `a` dominates a predecessor of `b`, so `b` is a dominance frontier of `a`
#         df.add(test_with_bb)
#     return df

# One-liner version of the above function
def get_dom_frontier(dom: Dict[BasicBlock, Set[BasicBlock]], bb: BasicBlock) -> Set[BasicBlock]:
    return set([test_with_bb for test_with_bb in dom.keys() if (bb not in dom[test_with_bb] or bb == test_with_bb) and (bb in set.union(*[dom[each_tbp] for each_tbp in test_with_bb.pred] or [set()]))])


In [7]:
print(f"{bscript}")
print()
for each_func, bbs in app_graph.items():
    _max_bb_label_len = max([len(bb.label.label) for bb in bbs])
    print(each_func)
    dom = generate_dom_dict(bbs)
    for bb in bbs:
        print(f"{bb.label.label.ljust(_max_bb_label_len)} : {', '.join([x.label.label for x in sorted(get_dom_frontier(dom, bb))])}")
    print()

BrilScript ::	loopcond.bril <1 func> @ ../bril/examples/test/dom

BrilFunction ::	main (  ) -> None: <22 instr>
ENTRY  : 
entry  : 
loop   : loop
body   : loop
then   : endif
endif  : loop
exit   : 
RETURN : 



In [8]:
def get_style_attrs(bb: BasicBlock) -> dict:
    _addi_attrs = dict()
    if bb.label.label in [ENTRY_POINT_NAME, RETURN_POINT_NAME]:
        _addi_attrs['shape'] = 'doublecircle'
        _addi_attrs['style'] = 'filled'
        _addi_attrs['fillcolor'] = 'lightgray'
    else:
        _addi_attrs['shape'] = 'box'
    return _addi_attrs

def create_graphviz_dot_node(dot: gv.Digraph, func: bm.BrilFunction, bb: BasicBlock, **kw_args) -> None:
    dot.node(f"FUNC_{func.name}_NODE_{bb.label.label}", label=f"{bb}", **get_style_attrs(bb), **kw_args)

def create_graphviz_dot_edge(dot: gv.Digraph, func: bm.BrilFunction, from_bb: BasicBlock, to_bb: BasicBlock, **kw_args) -> None:
    dot.edge(f"FUNC_{func.name}_NODE_{from_bb.label.label}", f"FUNC_{func.name}_NODE_{to_bb.label.label}", **kw_args)

def create_graphviz_dot(app_graph: Dict[bm.BrilFunction, List[BasicBlock]]) -> Tuple[gv.Digraph, gv.Digraph]:
    dot_cfg = gv.Digraph('CFG', comment='Control Flow Graph')
    dot_domt = gv.Digraph('DOMTREE', comment='Dominator Tree')
    for each_func, bbs in app_graph.items():
        
        print(each_func)
        dom = generate_dom_dict(bbs)
        _max_bb_label_len = max([len(bb.label.label) for bb in bbs])

        # each func is a subgraph
        with dot_cfg.subgraph(name=f"cluster_{each_func.name}") as dot_cfg_func, dot_domt.subgraph(name=f"cluster_{each_func.name}") as dot_domt_func:
            dot_cfg_func.attr(label=each_func.name)
            dot_cfg_func.attr(color='#f7f7f7')
            dot_cfg_func.attr(style='filled')
            dot_cfg_func.attr(rankdir='TB')
            dot_domt_func.attr(label=each_func.name)
            dot_domt_func.attr(color='#f7f7f7')
            dot_domt_func.attr(style='filled')
            dot_domt_func.attr(rankdir='TB')

            # generate cfg dot graph
            for each_bb in bbs:
                dom_frontiers = get_dom_frontier(dom, each_bb)
                create_graphviz_dot_node(dot_cfg_func, each_func, each_bb)
                for each_succ in each_bb.succ:
                    create_graphviz_dot_edge(dot_cfg_func, each_func, each_bb, each_succ)
                for each_df in dom_frontiers:
                    create_graphviz_dot_edge(dot_cfg_func, each_func, each_bb, each_df, style='dashed', color='#66ccff')

            # generate domt dot graph
            for each_bb in dom.keys():
                least_superset_dom = get_lease_superset_dom(dom, each_bb)
                print(f"  {each_bb.label.label.ljust(_max_bb_label_len)} <- {(least_superset_dom.label.label if least_superset_dom else '[ROOT]').ljust(_max_bb_label_len)} : {', '.join([x.label.label for x in sorted(dom[each_bb])])}")
                create_graphviz_dot_node(dot_domt_func, each_func, each_bb)
                if least_superset_dom:
                    create_graphviz_dot_edge(dot_domt_func, each_func, least_superset_dom, each_bb)
            
            print()
    return dot_cfg, dot_domt


print(f"{bscript}")
print()
dot_cfg, dot_domt = create_graphviz_dot(app_graph)
dot_cfg.render(os.path.join(SAVE_DIR, bscript.script_name+".cfg.gv"), view=False)
dot_domt.render(os.path.join(SAVE_DIR, bscript.script_name+".domt.gv"), view=False)


BrilScript ::	loopcond.bril <1 func> @ ../bril/examples/test/dom

BrilFunction ::	main (  ) -> None: <22 instr>
  ENTRY  <- [ROOT] : ENTRY
  entry  <- ENTRY  : ENTRY, entry
  loop   <- entry  : ENTRY, entry, loop
  body   <- loop   : ENTRY, body, entry, loop
  then   <- body   : ENTRY, body, entry, loop, then
  endif  <- body   : ENTRY, body, endif, entry, loop
  exit   <- loop   : ENTRY, entry, exit, loop
  RETURN <- exit   : ENTRY, entry, exit, loop, RETURN



'save/loopcond.bril.domt.gv.pdf'

In [9]:
print_blocks(bscript)

BrilFunction ::	main (  ) -> None: <22 instr>
block "entry":
  BrilInstruction_Const ::	x (int) <- [const] 0
  BrilInstruction_Const ::	i (int) <- [const] 0
  BrilInstruction_Const ::	one (int) <- [const] 1
block "loop":
  BrilInstruction_Const ::	max (int) <- [const] 10
  BrilInstruction_ValOp ::	cond (bool) <- [lt] i, max 
  BrilInstruction_EffOp ::	[br] cond {body, exit} 
block "body":
  BrilInstruction_Const ::	mid (int) <- [const] 5
  BrilInstruction_ValOp ::	cond (bool) <- [lt] i, mid 
  BrilInstruction_EffOp ::	[br] cond {then, endif} 
block "then":
  BrilInstruction_ValOp ::	x (int) <- [add] x, one 
  BrilInstruction_EffOp ::	[jmp]  {endif} 
block "endif":
  BrilInstruction_Const ::	factor (int) <- [const] 2
  BrilInstruction_ValOp ::	x (int) <- [mul] x, factor 
  BrilInstruction_ValOp ::	i (int) <- [add] i, one 
  BrilInstruction_EffOp ::	[jmp]  {loop} 
block "exit":
  BrilInstruction_EffOp ::	[print] x 

